In [2]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
from torch.utils.data import DataLoader

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

In [4]:
UNK_IDX = 2
PAD_IDX = 3
SOS_token = 0
EOS_token = 1

In [5]:
def read_dataset(file):
    f = open(file)
    list_l = []
    for line in f:
        list_l.append(line.strip())
    df = pd.DataFrame()
    df['data'] = list_l
    return df

In [6]:
en_train = read_dataset("./iwslt-vi-en/train.tok.en")
en_val = read_dataset("./iwslt-vi-en/dev.tok.en")

In [7]:
vi_train = read_dataset("./iwslt-vi-en/train.tok.vi")
vi_val = read_dataset("./iwslt-vi-en/dev.tok.vi")

In [8]:
print(en_train.shape,vi_train.shape)
print(en_val.shape,vi_val.shape)

(133317, 1) (133317, 1)
(1268, 1) (1268, 1)


In [9]:
train = pd.DataFrame()
train['en_data'] = en_train['data']
train['vi_data'] = vi_train['data']

In [10]:
train.ix[38600]

/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


en_data                  
vi_data    " tán_thưởng "
Name: 38600, dtype: object

In [11]:
val = pd.DataFrame()
val['en_data'] = en_val['data']
val['vi_data'] = vi_val['data']

In [12]:
print(train.shape,val.shape)

(133317, 2) (1268, 2)


In [13]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UKN",3:"PAD"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word.lower())

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [14]:
# train['en_data'] = train['en_data'].apply(lambda x: x.lower())
# train['vi_data'] = train['vi_data'].apply(lambda x:x.lower())

In [15]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


In [16]:
en_lang = Lang("en")
for ex in train['en_data']:
#     ex = unicodeToAscii(ex)
#     ex = re.sub(r"[^a-zA-Z.!?]+", r" ", ex)
    en_lang.addSentence(ex)

In [17]:
vi_lang = Lang("vi")
for ex in train['vi_data']:
#     ex = unicodeToAscii(ex)
#     ex = re.sub(r"[^a-zA-Z.!?_]+", r" ", ex)
    vi_lang.addSentence(ex)

In [18]:
# vi_lang.word2index

In [19]:
def split(df):
    df['en_tokenized'] = df["en_data"].apply(lambda x:x.lower().split( ))
    df['vi_tokenized'] = df['vi_data'].apply(lambda x:x.lower().split( ))
    return df

In [20]:
train = split(train)

In [21]:
val = split(val)

In [22]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]"
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ..."
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà..."
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,..."


In [23]:
# train[train['vi_len']==713]['en_data']

In [24]:
# vi_lang.word2index

In [25]:
def token2index_dataset(df):
    for lan in ['en','vi']:
        indices_data = []
        if lan=='en':
            lang_obj = en_lang
        else:
            lang_obj = vi_lang
        for tokens in df[lan+'_tokenized']:
            index_list = [lang_obj.word2index[token] if token in lang_obj.word2index else UNK_IDX for token in tokens]
            index_list.append(EOS_token)
#             index_list.insert(0,SOS_token)
            indices_data.append(index_list)
        df[lan+'_idized'] = indices_data
    return df

In [26]:
train = token2index_dataset(train)

In [27]:
val = token2index_dataset(val)

In [28]:
class Vietnamese(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx]['en_idized']
        viet = self.df.iloc[idx]['vi_idized']
        en_len = self.df.iloc[idx]['en_len']
        vi_len = self.df.iloc[idx]['vi_len']
        return [english,viet,en_len,vi_len]

In [29]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized,en_idized,vi_idized
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 1]","[4, 5, 6, 7, 8, 9, 1]"
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q...","[13, 14, 15, 16, 17, 18, 4, 5, 19, 10, 20, 21,...","[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 8..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ...","[47, 48, 49, 50, 51, 50, 52, 53, 54, 7, 55, 21...","[52, 53, 38, 54, 55, 56, 8, 57, 58, 30, 21, 22..."
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà...","[26, 56, 62, 49, 63, 64, 65, 66, 50, 67, 29, 1...","[67, 21, 61, 68, 69, 70, 71, 72, 8, 25, 9, 13,..."
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,...","[65, 73, 74, 75, 76, 21, 7, 77, 78, 21, 17, 8,...","[80, 81, 82, 83, 6, 84, 30, 26, 6, 85, 10, 86,..."


In [30]:
train['en_len'] = train['en_idized'].apply(lambda x: len(x))
train['vi_len'] = train['vi_idized'].apply(lambda x:len(x))

In [31]:
val['en_len'] = val['en_idized'].apply(lambda x: len(x))
val['vi_len'] = val['vi_idized'].apply(lambda x: len(x))

In [32]:
train['en_len'].describe()

count    133317.000000
mean         21.299399
std          15.035268
min           1.000000
25%          11.000000
50%          17.000000
75%          27.000000
max         629.000000
Name: en_len, dtype: float64

In [33]:
train = train[np.logical_and(train['en_len']>=2,train['vi_len']>=2)]

In [34]:
train = train[train['en_len']<=30]

In [35]:
val = val[np.logical_and(val['en_len']>=2,val['vi_len']>=2)]

In [36]:
val = val[val['en_len']<=30]

In [37]:
MAX_LEN = 31
def vocab_collate_func(batch):
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []

    for datum in batch:
        en_len.append(datum[2])
        vi_len.append(datum[3])
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s1)
        vi_data.append(padded_vec_s2)
        
    return [torch.from_numpy(np.array(vi_data)),torch.from_numpy(np.array(en_data)),
            torch.from_numpy(np.array(vi_len)), torch.from_numpy(np.array(en_len))]

In [38]:
bs = 10
transformed_dataset = {'train': Vietnamese(train),
                       'validate': Vietnamese(val)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs, collate_fn=vocab_collate_func,
                        shuffle=True, num_workers=0) for x in ['train', 'validate']}

In [39]:
data = next(iter(dataloader['train']))

In [40]:
data[0].size()

torch.Size([10, 31])

In [41]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,bi):
        super(EncoderRNN, self).__init__()
        self.bi = bi
        if self.bi:
            self.mul = 2
        else:
            self.mul = 1
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
#         bss = input.size(0)
        output = embedded
#         print("emb size",output.size())
        output, hidden = self.gru(output, hidden)
#         print("rnn out",output.size())
#         print("rnn hid",hidden.size())
        return output, hidden

    def initHidden(self,bs):
        return torch.zeros(self.mul, bs, self.hidden_size).cuda()

In [42]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,bi):
        super(DecoderRNN, self).__init__()
        self.bi = bi
        if self.bi:
            self.mul=2
        else:
            self.mul=1
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,batch_first=True,bidirectional=self.bi)
        self.out = nn.Linear(self.mul * hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
#         bss = input.size(0)
#         print(input.size(),bss)
#         print(bss)
#         print("in",input.size())
#         print("hid",hidden.size())
        
        output = self.embedding(input)
#         .view(bss,-1,self.hidden_size)
#         print("ou",output.size())
        output = F.relu(output)
#         print("ou rel",output.size())
        output, hidden = self.gru(output, hidden)
#         print("out",output.size(),"hid",hidden.size())
        output = self.softmax(self.out(output))
#         print("sm",output.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.mul, bs, self.hidden_size).cuda()

In [43]:
import random
def encode_decode(encoder,decoder,data_en,data_de):
    use_teacher_forcing = True if random.random() < 0.5 else False
#     use_teacher_forcing = True
    bss = data_en.size(0)
#     print("data de",data_de.size())
    en_h = encoder.initHidden(bss)
    en_out,en_hid = encoder(data_en,en_h)
    
    decoder_hidden = en_hid
    decoder_input = torch.tensor([[SOS_token]]*bss).cuda()

    if use_teacher_forcing:
        d_out = []
        for i in range(31):
            decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
            d_out.append(decoder_output)
            decoder_input = data_de[:,i].view(-1,1)
#             print(decoder_input.size())
        d_hid = decoder_hidden
        d_out = torch.cat(d_out,dim=1)
#         d_out,d_hid = decoder(decoder_input,decoder_hidden)  
#         d_out,d_hid = decoder(data_de,d_hid)
    else:
        d_out = []
        for i in range(31):
            decoder_output, decoder_hidden = decoder(decoder_input,decoder_hidden)
            d_out.append(decoder_output)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
        d_hid = decoder_hidden
        d_out = torch.cat(d_out,dim=1)
#         print(len(d_out),d_out[0].size(),torch.cat(d_out,dim=1).size())
    return d_out, d_hid

In [44]:
def train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, loss_fun, num_epochs=60):
    best_score = 0
    best_au = 0
    loss_hist = {'train': [], 'validate': []}
    acc_hist = {'train': [], 'validate': []}
    for epoch in range(num_epochs):
        for ex, phase in enumerate(['train', 'validate']):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train(True)
                decoder.train(True)
            else:
                encoder.train(False)
                decoder.train(False)
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_i = data[0].cuda()
                decoder_i = data[1].cuda()
                                
                out, hidden = encode_decode(encoder,decoder,encoder_i,decoder_i)
#                 print(out.size())
#                 print(decoder_i.size())
#                 _, top1_predicted = torch.max(out, dim=2)
#                 print(out.float().view(-1,out.size(-1)).size(),decoder_i.long().view(-1).size())
#                 print(top1_predicted.float().view(-1).size())
                loss = loss_fun(out.float().view(-1,out.size(-1)), decoder_i.long().view(-1))
                N = decoder_i.size(0)
                running_loss += loss.item() * N
                
                total += N
                
#                 _, top1_predicted = torch.max(y_out, dim=1)
#                 top1_correct += int((top1_predicted == y).sum())

                if phase == 'train':
                    loss.backward()
                    encoder_optimizer.step()
                    decoder_optimizer.step()

#                 running_total += N
            epoch_loss = running_loss / total
#             epoch_acc = top1_correct / total
            epoch_acc = 0
            loss_hist[phase].append(epoch_loss)
            acc_hist[phase].append(epoch_acc)
            print("epoch {} {} loss = {}, accurancy = {} time = {}".format(epoch, phase, epoch_loss, epoch_acc,
                                                                           time.time() - start))
        if phase == 'validate' and epoch_acc > best_score:
            best_score = epoch_acc
#             torch.save(model, save_dir+save_name+str(n_channel)+str(n_top)+str(vocab_size))
    print("Training completed. Best accuracy is {}".format(best_score))
    return encoder,decoder


In [45]:
device='gpu'

In [46]:
learning_rate = 1e-4
bi=True
encoder = EncoderRNN(vi_lang.n_words,100,bi).cuda()
decoder = DecoderRNN(100,en_lang.n_words,bi).cuda()
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [47]:
enc, dec = train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, criterion, num_epochs = 50)

epoch 0 train loss = 6.940234226582324, accurancy = 0 time = 656.2534871101379
epoch 0 validate loss = 6.054321828553287, accurancy = 0 time = 2.4726788997650146


KeyboardInterrupt: 

In [164]:
data = next(iter(dataloader['validate']))
out,hid = encode_decode(enc,dec,data[0][1].view(1,-1).cuda(),data[1][1].view(1,-1).cuda())
_, top1_predicted = torch.max(out, dim=2)

In [167]:
given=[]
for i in data[1][1]:
    given.append(vi_lang.index2word[i.item()])

In [168]:
pred = []
for i in top1_predicted[0]:
#     print(i.item())
    pred.append(vi_lang.index2word[i.item()])

In [169]:
given

['đây',
 'nên',
 'đáng',
 'là',
 'nguyên_tắc',
 'đầu_tiên',
 'về',
 'viện_trợ',
 '.',
 'EOS',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD']

In [ ]:
pred